# Reviews Text Sentiment Analysis For Thailand Tourism Recommendation

This project target for create recommendation system from rating thailand tourism by learning review positive or negative

#### Step #1 - Setup Project Library Package

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import os
from geopy.geocoders import Nominatim
from dotenv import load_dotenv

load_dotenv()

True

#### Step #2 - Data Preparation
##### 2.1 Thailand Province -> Amphure Data <br>
https://github.com/kongvut/thai-province-data

In [15]:
province_data = pd.read_json('hook/data/api_province_with_amphure_tambon.json')
province_data

,id,name_th,name_en,geography_id,created_at,updated_at,deleted_at,amphure
0,1,กรุงเทพมหานคร,Bangkok,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1001, 'name_th': 'เขตพระนคร', 'name_en..."
1,2,สมุทรปราการ,Samut Prakan,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1101, 'name_th': 'เมืองสมุทรปราการ', '..."
2,3,นนทบุรี,Nonthaburi,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1201, 'name_th': 'เมืองนนทบุรี', 'name..."
3,4,ปทุมธานี,Pathum Thani,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1301, 'name_th': 'เมืองปทุมธานี', 'nam..."
4,5,พระนครศรีอยุธยา,Phra Nakhon Si Ayutthaya,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1401, 'name_th': 'พระนครศรีอยุธยา', 'n..."
...,...,...,...,...,...,...,...,...
72,73,พัทลุง,Phatthalung,6,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 9301, 'name_th': 'เมืองพัทลุง', 'name_..."
73,74,ปัตตานี,Pattani,6,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 9401, 'name_th': 'เมืองปัตตานี', 'name..."
74,75,ยะลา,Yala,6,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 9501, 'name_th': 'เมืองยะลา', 'name_en..."
75,76,นราธิวาส,Narathiwat,6,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 9601, 'name_th': 'เมืองนราธิวาส', 'nam..."


In [8]:
provinces = province_data['name_en'].tolist()
geolocator = Nominatim(user_agent="thailand_provinces")
data = {'Province': [], 'Latitude': [], 'Longitude': []}

for province in provinces:
    location = geolocator.geocode(f"{province}, Thailand")
    if location:
        data['Province'].append(province)
        data['Latitude'].append(location.latitude)
        data['Longitude'].append(location.longitude)

thailland_province_coordinates = pd.DataFrame(data)


In [4]:
# thailland_province_coordinates.to_csv("thailand_province_coordinate.csv", index=False, header=True, sep=',', encoding='utf-8')

In [4]:
thailland_province_coordinates[thailland_province_coordinates['Province'] == 'Bangkok']

,Province,Latitude,Longitude
0,Bangkok,13.752494,100.493509


 ##### 2.2 Thailand Tourism Location API Data <br>
https://developers.tourismthailand.org/

In [26]:
import hook.thai_tourism_data as tat
tourism_location = tat.ThaiTourismAPI("EN")

def getTourismLocationsByProvince(province_name, province_location, category):
    search_params = {
            "keyword": "",  # Example keyword for Kanchanaburi
            "location": province_location,
            "categorycodes": category,
            "province_name": province_name,
            "radius": 1000,
            "number_of_result": 100,
            "page_number": 1,
            "destination": province_name,
            "filter_by_update_date": "2019/09/01-2024/05/31"
        }
    tourism_data = tourism_location.fetch_search_location(**search_params)

    df = pd.DataFrame([vars(place) for place in tourism_data])
    return df


In [17]:
thailland_province_coordinates = pd.read_csv("dataset/thailand_province_coordinate.csv")

In [27]:
tourism_attraction_locations = pd.DataFrame()
tourism_hotels_locations = pd.DataFrame()
tourism_restaurants_locations = pd.DataFrame()
tourism_shops_locations = pd.DataFrame()
# CATEGORIES: ATTRACTION | ACCOMMODATION  | RESTAURANT | GEOS | SHOP

for index, pvn in province_data.iterrows():
    coordinate = thailland_province_coordinates[thailland_province_coordinates['Province'] == pvn['name_en']]
    
    if len(coordinate['Latitude'].values) > 0:
        # tourism_shops_location = getTourismLocationsByProvince(pvn['name_th'], 
        # f"{coordinate['Latitude'].values[0]},"
        # f"{coordinate['Longitude'].values[0]}",
        # "SHOP")
        
        # tourism_shops_locations = pd.concat([tourism_shops_locations, tourism_shops_location], ignore_index=True)
        
        province_torism_location = getTourismLocationsByProvince(pvn['name_en'], 
        f"{coordinate['Latitude'].values[0]},"
        f"{coordinate['Longitude'].values[0]}",
        "ATTRACTION")
        
        tourism_attraction_locations = pd.concat([tourism_attraction_locations, province_torism_location], ignore_index=True)
        
        # tourism_hotels_location = getTourismLocationsByProvince(pvn['name_en'], 
        # f"{coordinate['Latitude'].values[0]},"
        # f"{coordinate['Longitude'].values[0]}",
        # "ACCOMMODATION")
        
        # tourism_hotels_locations = pd.concat([tourism_hotels_locations, tourism_hotels_location], ignore_index=True)
        
        # tourism_restaurants_location = getTourismLocationsByProvince(pvn['name_th'], 
        # f"{coordinate['Latitude'].values[0]},"
        # f"{coordinate['Longitude'].values[0]}",
        # "RESTAURANT")
        
        # tourism_restaurants_locations = pd.concat([tourism_restaurants_locations, tourism_restaurants_location], ignore_index=True)
        

In [ ]:
tourism_shops_locations

In [28]:
tourism_attraction_locations

,place_id,place_name,latitude,longitude,category_code,category_description,sha_name,sha_type_code,sha_type_description,sha_cate_id,...,address,sub_district,district,province,postcode,thumbnail_url,destination,tags,distance,update_date
0,P03013220,Wat Ratchabophit Sathitmahasimaram,13.749132,100.497341,ATTRACTION,Attraction,,,,,...,2 Fueang Nakhon Road,Wat Ratchabophit,Phra Nakhon,Bangkok,10200,https://tatapi.tourismthailand.org/tatfs/Image...,BANGKOK,None,0.0,2024-05-27T14:30:11.51
1,P03013472,Wat Yan Nawa,13.717333,100.513552,ATTRACTION,Attraction,,,,,...,40 Charoen Krung Road,Yan Nawa,Sathon,Bangkok,10120,https://tatapi.tourismthailand.org/tatfs/Image...,BANGKOK,None,0.0,2024-05-24T15:09:37.447
2,P03013449,Tiger God Shrine,13.753852,100.498319,ATTRACTION,Attraction,,,,,...,468 Tanao Road,San Chaopho Suea,Phra Nakhon,Bangkok,10200,https://tatapi.tourismthailand.org/tatfs/Image...,BANGKOK,None,0.0,2024-05-24T15:04:50.82
3,P03013827,Wat Thewarat Kunchorn Worawiharn,13.772303,100.501957,ATTRACTION,Attraction,,,,,...,,Wachira Phayaban,Dusit,Bangkok,10300,https://tatapi.tourismthailand.org/tatfs/Image...,BANGKOK,None,0.0,2024-05-24T14:40:56.073
4,P03013469,Wat Apson Sawan Worawihan,13.720030,100.469886,ATTRACTION,Attraction,,,,,...,,Pak Khlong Phasi Charoen,Phasi Charoen,Bangkok,10160,https://tatapi.tourismthailand.org/tatfs/Image...,BANGKOK,None,0.0,2024-05-24T14:33:22.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3636,P03017307,Tham Fun Waterfall,18.266144,103.903480,ATTRACTION,Attraction,,,,,...,,Nong Doen,Bung Khla,Bueng Kan,38000,https://tatapi.tourismthailand.org/tatfs/Image...,BUENG KAN,None,0.0,2020-02-19T18:21:32.91
3637,P03012250,Pu Ya Shrine,18.371745,103.649444,ATTRACTION,Attraction,,,,,...,,Wisit,Mueang Bueng Kan,Bueng Kan,38000,https://tatapi.tourismthailand.org/tatfs/IMAGE...,Bueng Kan,None,0.0,2020-02-13T08:00:00
3638,P03012246,Kut Thing Yai,18.296982,103.662169,ATTRACTION,Attraction,,,,,...,,Non Sombun,Mueang Bueng Kan,Bueng Kan,38000,https://tatapi.tourismthailand.org/tatfs/IMAGE...,Bueng Kan,None,0.0,2020-02-13T08:00:00
3639,P03012247,Local Weaving Group Ban Sa Ngor,18.443938,103.416335,ATTRACTION,Attraction,,,,,...,,Ho Kham,Mueang Bueng Kan,Bueng Kan,38000,https://tatapi.tourismthailand.org/tatfs/Image...,Bueng Kan,None,0.0,2020-02-13T08:00:00


In [ ]:
tourism_hotels_locations

In [ ]:
tourism_restaurants_locations

In [29]:
# tourism_shops_locations.to_csv("tourism_shops_locations.csv", index=False, na_rep='N/A', header=True, sep=',', encoding='utf-8')

tourism_attraction_locations.to_csv("tourism_attraction_locations_en.csv", index=False, na_rep='N/A', header=True, sep=',', encoding='utf-8')

# tourism_hotels_locations.to_csv("tourism_hotels_locations.csv", index=False, na_rep='N/A', header=True, sep=',', encoding='utf-8')

# tourism_restaurants_locations.to_csv("tourism_restaurants_locations.csv", index=False, na_rep='N/A', header=True, sep=',', encoding='utf-8')

In [31]:
tourism_attraction_locations = pd.read_csv("tourism_attraction_locations_en.csv")
tourism_attraction_locations

,place_id,place_name,latitude,longitude,category_code,category_description,sha_name,sha_type_code,sha_type_description,sha_cate_id,...,address,sub_district,district,province,postcode,thumbnail_url,destination,tags,distance,update_date
0,P03013220,Wat Ratchabophit Sathitmahasimaram,13.749132,100.497341,ATTRACTION,Attraction,NaN,NaN,NaN,NaN,...,2 Fueang Nakhon Road,Wat Ratchabophit,Phra Nakhon,Bangkok,10200.0,https://tatapi.tourismthailand.org/tatfs/Image...,BANGKOK,NaN,0.0,2024-05-27T14:30:11.51
1,P03013472,Wat Yan Nawa,13.717333,100.513552,ATTRACTION,Attraction,NaN,NaN,NaN,NaN,...,40 Charoen Krung Road,Yan Nawa,Sathon,Bangkok,10120.0,https://tatapi.tourismthailand.org/tatfs/Image...,BANGKOK,NaN,0.0,2024-05-24T15:09:37.447
2,P03013449,Tiger God Shrine,13.753852,100.498319,ATTRACTION,Attraction,NaN,NaN,NaN,NaN,...,468 Tanao Road,San Chaopho Suea,Phra Nakhon,Bangkok,10200.0,https://tatapi.tourismthailand.org/tatfs/Image...,BANGKOK,NaN,0.0,2024-05-24T15:04:50.82
3,P03013827,Wat Thewarat Kunchorn Worawiharn,13.772303,100.501957,ATTRACTION,Attraction,NaN,NaN,NaN,NaN,...,NaN,Wachira Phayaban,Dusit,Bangkok,10300.0,https://tatapi.tourismthailand.org/tatfs/Image...,BANGKOK,NaN,0.0,2024-05-24T14:40:56.073
4,P03013469,Wat Apson Sawan Worawihan,13.720030,100.469886,ATTRACTION,Attraction,NaN,NaN,NaN,NaN,...,NaN,Pak Khlong Phasi Charoen,Phasi Charoen,Bangkok,10160.0,https://tatapi.tourismthailand.org/tatfs/Image...,BANGKOK,NaN,0.0,2024-05-24T14:33:22.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3636,P03017307,Tham Fun Waterfall,18.266144,103.903480,ATTRACTION,Attraction,NaN,NaN,NaN,NaN,...,NaN,Nong Doen,Bung Khla,Bueng Kan,38000.0,https://tatapi.tourismthailand.org/tatfs/Image...,BUENG KAN,NaN,0.0,2020-02-19T18:21:32.91
3637,P03012250,Pu Ya Shrine,18.371745,103.649444,ATTRACTION,Attraction,NaN,NaN,NaN,NaN,...,NaN,Wisit,Mueang Bueng Kan,Bueng Kan,38000.0,https://tatapi.tourismthailand.org/tatfs/IMAGE...,Bueng Kan,NaN,0.0,2020-02-13T08:00:00
3638,P03012246,Kut Thing Yai,18.296982,103.662169,ATTRACTION,Attraction,NaN,NaN,NaN,NaN,...,NaN,Non Sombun,Mueang Bueng Kan,Bueng Kan,38000.0,https://tatapi.tourismthailand.org/tatfs/IMAGE...,Bueng Kan,NaN,0.0,2020-02-13T08:00:00
3639,P03012247,Local Weaving Group Ban Sa Ngor,18.443938,103.416335,ATTRACTION,Attraction,NaN,NaN,NaN,NaN,...,NaN,Ho Kham,Mueang Bueng Kan,Bueng Kan,38000.0,https://tatapi.tourismthailand.org/tatfs/Image...,Bueng Kan,NaN,0.0,2020-02-13T08:00:00


##### 2.3 TripAdvisor API Data <br>
https://tripadvisor-content-api.readme.io/reference/getlocationdetails

In [4]:
from hook.tripadvisor_data import TripAdvisorApi

load_dotenv()

# Ensure the API key is set
api_key = os.getenv('TRIPADVISOR_API_KEY')

# Initialize the API client
thai_tourism_api = TripAdvisorApi(api_key, "EN")
# print(thai_tourism_api)

# Test API request
response = thai_tourism_api.test_api_request()

Testing Tripadvisor API... FDB8183A632245CDA0EC0DBF74159030
{
  "data": [
    {
      "location_id": "317608",
      "name": "สะพานข้ามแม่น้ำแคว",
      "address_obj": {
        "street1": "ต. ท่ามะขาม",
        "street2": "",
        "city": "เมืองกาญจนบุรี",
        "state": "จังหวัดกาญจนบุรี",
        "country": "ไทย",
        "postalcode": "71000",
        "address_string": "ต. ท่ามะขาม, เมืองกาญจนบุรี 71000 ไทย"
      }
    },
    {
      "location_id": "6737314",
      "name": "วัดถ้ำเสือ",
      "address_obj": {
        "street1": "Moo 3",
        "street2": "",
        "city": "ท่าม่วง",
        "state": "จังหวัดกาญจนบุรี",
        "country": "ไทย",
        "postalcode": "71110",
        "address_string": "Moo 3, ท่าม่วง 71110 ไทย"
      }
    },
    {
      "location_id": "554151",
      "name": "แม่น้ำแคว",
      "address_obj": {
        "street1": "",
        "street2": "",
        "city": "เมืองกาญจนบุรี",
        "state": "จังหวัดกาญจนบุรี",
        "country": "ไทย",
     

In [5]:
search_tripadvisor_location = {
    "searchQuery": 'สะพานพระราม', 
    "category": "attractions", 
    "phone": None, 
    "address": "thailand",
    "latLong": None, 
    "radius": 1000, 
    "radiusUnit": 'km',
    "language": "en"
} 

response = thai_tourism_api.location_search(search_tripadvisor_location)
# response.json()['data'][0]["location_id"]
location = response.json()['data']
df_location = pd.concat(pd.DataFrame({"location_id": location['location_id'], "name": location['name']}))
df_location

<Response [200]>

In [35]:
location

{'location_id': '16735656',
 'name': 'Address Sky View',
 'address_obj': {'street1': 'Sheikh Mohammed Bin Rashed Boulevard',
  'city': 'Dubai',
  'state': 'Emirate of Dubai',
  'country': 'United Arab Emirates',
  'address_string': 'Sheikh Mohammed Bin Rashed Boulevard, Dubai United Arab Emirates'}}

In [37]:

pd.DataFrame(pd.json_normalize(location))

,location_id,name,address_obj.street1,address_obj.city,address_obj.state,address_obj.country,address_obj.address_string
0,16735656,Address Sky View,Sheikh Mohammed Bin Rashed Boulevard,Dubai,Emirate of Dubai,United Arab Emirates,"Sheikh Mohammed Bin Rashed Boulevard, Dubai Un..."


In [35]:
# tourism_attraction_locations
tripadvisor_attraction_locations = pd.DataFrame(columns=["location_id", "name"])

for index, attraction in tourism_attraction_locations.iterrows():
    search_tripadvisor_location = {
        "searchQuery": attraction['place_name'], 
        "category": "attractions", 
        "phone": None, 
        "address": "thailand",
        "latLong": None, 
        "radius": 1000, 
        "radiusUnit": 'km',
        "language": "th"
    } 

    response = thai_tourism_api.location_search(search_tripadvisor_location)
    
    location = response.json()['data'][0]
    
    print(f">>> #{index} {attraction['place_name']} ==> TripAdvisor [{location['name']}]")
    
    # Create a new DataFrame with the location_id and name
    new_entry = pd.DataFrame({"location_id": [location['location_id']], "name": [location['name']]})
    
    # Concatenate the new entry to the main DataFrame
    tripadvisor_attraction_locations = pd.concat([tripadvisor_attraction_locations, new_entry], ignore_index=True)
    
    if index == 5:
        break

>>> #0 Wat Ratchabophit Sathitmahasimaram ==> TripAdvisor [Wat Ratchabophit]
>>> #1 Wat Yan Nawa ==> TripAdvisor [Wat Phra Chetuphon]
>>> #2 Tiger God Shrine ==> TripAdvisor [Tiger God Shrine]
>>> #3 Wat Thewarat Kunchorn Worawiharn ==> TripAdvisor [Wat Intharam Worawiharn]
>>> #4 Wat Apson Sawan Worawihan ==> TripAdvisor [Wat Sothon Wararam Worawihan]
>>> #5 Wat Ratchaorasaram Ratchaworawihan ==> TripAdvisor [Wat Thepsirintrawat Ratchaworawihan]


In [36]:
tripadvisor_attraction_locations

,location_id,name
0,2209612,Wat Ratchabophit
1,311043,Wat Phra Chetuphon
2,7660023,Tiger God Shrine
3,9706798,Wat Intharam Worawiharn
4,1229768,Wat Sothon Wararam Worawihan
5,8595657,Wat Thepsirintrawat Ratchaworawihan


In [37]:
tripadvisor_attraction_locations.to_csv("TripAdvisor_Location_Ids.csv")

In [ ]:
# Import necessary libraries
import geopandas as gpd
import matplotlib.pyplot as plt

# Load the GeoJSON file
geojson_file = 'hook/data/thai_province_coordinates_geo.json'
gdf = gpd.read_file(geojson_file)

# Check for empty geometries and remove them
gdf = gdf[~gdf.is_empty]

# Ensure the CRS is set to WGS 84 (EPSG:4326)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)

# Initialize a Matplotlib figure and axis
fig, ax = plt.subplots(figsize=(12, 8))

# Plot all provinces in light blue
gdf.plot(ax=ax, color='lightblue', edgecolor='black')

# Highlight Bangkok in red, checking if it exists in the data
bangkok = gdf[gdf['name'] == 'Bangkok Metropolis']
if not bangkok.empty:
    bangkok.plot(ax=ax, color='red', edgecolor='black')

# Set the aspect of the plot to equal to ensure accurate representation
ax.set_aspect('equal')

# Add title and labels
ax.set_title('Map of Thai Provinces with Bangkok Highlighted', fontsize=15)
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)

# Show the plot
plt.show()
